In [1]:
import os
import time
import tqdm
import inceptionv3
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from albumentations import *
import utils
import defense

/home/eason/anaconda3/envs/untitled/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/eason/anaconda3/envs/untitled/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/eason/anaconda3/envs/untitled/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/eason/anaconda3/envs/untitled/lib/pyt

In [2]:
config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
sess = tf.Session(config=config)
data_path = "./data"
output_path = "./finalresults"
if not os.path.exists(output_path):
    os.makedirs(output_path)
cleandata = np.load(os.path.join(data_path, "clean100data.npy"))
cleanlabel = np.load(os.path.join(data_path, "clean100label.npy"))
targets = np.load(os.path.join(data_path, "random_targets.npy"))

In [22]:
xs = tf.placeholder(tf.float32, (299, 299, 3))
l2_x = tf.placeholder(tf.float32, (299, 299, 3))
l2_orig = tf.placeholder(tf.float32, (299, 299, 3))
label = tf.placeholder(tf.int32, ())
one_hot = tf.expand_dims(tf.one_hot(label, 1000), axis=0)

lam = 1e-6
epsilon = 0.05
max_steps = 50 #only extracting first 50 rounds of results
LR = 0.5

label = tf.placeholder(tf.int32, ())
one_hot = tf.expand_dims(tf.one_hot(label, 1000), axis=0)

logits, preds = inceptionv3.model(sess, tf.expand_dims(xs, axis=0))
l2_loss = tf.sqrt(2 * tf.nn.l2_loss(l2_x - l2_orig) /299/299/3)

labels = tf.tile(one_hot, (logits.shape[0], 1))
xent = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=labels))
loss = xent + lam * tf.maximum(l2_loss - epsilon, 0)
grad, = tf.gradients(loss, xs)

INFO:tensorflow:Scale of 0 disables regularizer.


In [ ]:
adv = np.copy(cleandata)



for index in tqdm.tqdm(range(cleandata.shape[0])):
    adv_pgd = np.copy(adv[index])
    num = 0
    l2 = utils.l2_distortion(adv_pgd,cleandata[index])
    while l2 < epsilon and num < 10000:
        g = sess.run(grad, {xs: adv_pgd, label: targets[index],l2_x: adv_pgd, l2_orig: cleandata[index]})
        adv_pgd -= LR * g
        adv_pgd = np.clip(adv_pgd, 0, 1)
        l2 = utils.l2_distortion(adv_pgd,cleandata[index])
        num = num + 1
    adv[index] = adv_pgd






  0%|          | 0/100 [00:00<?, ?it/s]



  1%|          | 1/100 [00:01<01:59,  1.20s/it]



  2%|▏         | 2/100 [03:58<1:57:37, 72.01s/it]



  3%|▎         | 3/100 [07:57<3:17:28, 122.15s/it]



  6%|▌         | 6/100 [07:57<2:13:58, 85.52s/it] 



  9%|▉         | 9/100 [07:57<1:30:48, 59.87s/it]



 11%|█         | 11/100 [07:57<1:02:11, 41.93s/it]



 15%|█▌        | 15/100 [07:58<41:35, 29.36s/it]  



 18%|█▊        | 18/100 [07:58<28:07, 20.58s/it]



 20%|██        | 20/100 [07:58<19:13, 14.42s/it]



 22%|██▏       | 22/100 [07:58<13:09, 10.12s/it]



 24%|██▍       | 24/100 [07:58<09:01,  7.12s/it]



 24%|██▍       | 24/100 [08:12<09:01,  7.12s/it]



 26%|██▌       | 26/100 [12:00<50:50, 41.22s/it]



 28%|██▊       | 28/100 [12:00<34:39, 28.88s/it]



 29%|██▉       | 29/100 [16:00<1:49:12, 92.28s/it]



 31%|███       | 31/100 [16:00<1:14:18, 64.62s/it]



 32%|███▏      | 32/100 [16:01<51:20, 45.30s/it]  



 35%|███▌      | 35/100 [16:01<34:22, 31.73s/it]



 3

In [21]:
l2

0.0034342869342475057

In [ ]:
np.save('AE/PGD_AE_data.npy', adv)